In [7]:
import pandas as pd
from sklearn.pipeline import Pipeline
import lib.transformers as tf

In [8]:
cwruData0 = pd.read_parquet('data/cwru0.parquet')
cwruData1 = pd.read_parquet('data/cwru1.parquet')

cwruData = pd.concat([cwruData0, cwruData1])

In [9]:
image_size = 56

selection_pipeline = \
Pipeline(steps=[
                ('DataSelector', tf.DataSelector(columns = ['fanEndData', 'driveEndData'], column_values = {'condition': ['Normal Baseline'], 'sampleRate': [12000]})),
                ('ArrayFlattener', tf.ArrayFlattener()),
                ('ArrayEqualizer', tf.ArrayEqualizer()),
                ('ArrayChunker', tf.ArrayChunker(image_size**2)),
                ('ArrayFlattener2', tf.ArrayFlattener()),
                ('ArrayReshaper', tf.ArrayReshaper((1, image_size, image_size)))
               ])

chunked_normal_data = selection_pipeline.transform(cwruData)

In [11]:
chunked_normal_data[0]

array([[[ 0.14566727,  0.09779637,  0.05485636, ..., -0.07458   ,
          0.02712   ,  0.12882   ],
        [ 0.18367636,  0.16169272,  0.09492   , ..., -0.19353819,
         -0.19682546, -0.16272   ],
        [-0.06143091,  0.03636545,  0.08731818, ...,  0.06533454,
          0.0113    , -0.04150182],
        ...,
        [-0.08156545, -0.12286182, -0.14053091, ...,  0.03698182,
          0.08916727,  0.12080728],
        [ 0.1269709 ,  0.10868546,  0.05095273, ...,  0.05773273,
          0.04437818,  0.02958545],
        [ 0.05608909,  0.05629455,  0.02732546, ...,  0.04746   ,
          0.00020545, -0.06245818]]], dtype=float32)

In [ ]:
import scipy.signal as signal
from sklearn.preprocessing import normalize
from PIL import Image

x = selection_pipeline.transform(cwruData)[0]
x = np.fft.fft(x)
x = np.abs(x) / len(x)
x = x[range(int(x.shape[0] / 2))]
x = x.reshape(-1,1)
print(x.shape)


def STFT(fl):
    f, t, Zxx = signal.stft(fl, nperseg=32)
    img = normalize(np.abs(Zxx) / len(Zxx))
    return img

stft_inp = np.array([np.array(Image.fromarray(STFT(array)).resize((32, 32))).reshape(1 , 32, 32) for array in selection_pipeline.transform(cwruData)], dtype = np.float32)